In [1]:
!pip install dagshub
!pip install mlflow

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.9/247.9 kB 5.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 69.3 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 3.5 MB/s eta 0:00:00
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10487 sha256=2fde113b8c1eee4499bb33894f8b549c14101e2bcd176e6ba4c901a7a9bcee04
  Stored in directory: /root/.cache/pip/wheels/c0/18/f6/f0d6be9d0435e2677ce5cc758e91da50053dce456a346f08c5
Successfully built fusepy
  Attempting uninstall: dacite
    Found existing installation: dacite 1.8.1
    Uninstalling dacite-1.8.1:
      Successfully uninstalled dacite-1.8.1
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalli

In [17]:
import dagshub
import mlflow
import os
import time
import numpy as np
import xgboost as xgb
import h2o
import glob
import gc
from pyngrok import ngrok
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from sklearn.metrics import roc_auc_score,precision_score,recall_score,f1_score,accuracy_score,log_loss

In [3]:
os.environ['MLFLOW_TRACKING_USERNAME'] = 'tnbmarketplace'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd7c1a451bfc46bf96fbbe5d723becf3955dfa2b3'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/tnbmarketplace/instacart_basket_analysis_exp_tracking.mlflow'
!dagshub login --token "d7c1a451bfc46bf96fbbe5d723becf3955dfa2b3"

2024-10-18 11:47:01,565 - [INFO]  - [_client.py:1026] - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"
✅ Token added successfully


In [4]:
dagshub.init(repo_name = "instacart_basket_analysis_exp_tracking",repo_owner = 'tnbmarketplace')

Accessing as tnbmarketplace

Initialized MLflow to track repo "tnbmarketplace/instacart_basket_analysis_exp_tracking"

Repository tnbmarketplace/instacart_basket_analysis_exp_tracking initialized!

In [5]:
!ngrok authtoken "2kvaYw5ZiG5bL8iM8YJBVJPk1Ru_3C16mMgmpKEBYb28PPLUe"
# Start Ngrok tunnel on H2O's port
ngrok_tunnel = ngrok.connect(54321)
print("H2O UI accessible at:", ngrok_tunnel)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                
H2O UI accessible at: NgrokTunnel: "https://4bf3-34-73-229-179.ngrok-free.app" -> "http://localhost:54321"


In [6]:
h2o.init(max_mem_size = 25)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu320.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu320.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpui78mvrd
  JVM stdout: /tmp/tmpui78mvrd/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpui78mvrd/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 18 days
H2O_cluster_name:,H2O_from_python_unknownUser_j7l7wg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,25 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [7]:
def get_time(start):
    return time.time() - start

In [8]:
exp_data = h2o.import_file('/kaggle/input/instacart-model-patcher/experiment_data')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
# exp_data['fold_no'] = exp_data['fold_no'].asfactor()
exp_data['reordered'] = exp_data['reordered'].asfactor() 

In [10]:
training_cols = exp_data.columns
training_cols.remove('reordered')
training_cols.remove('fold_no')

In [11]:
# logistic_model = H2OGeneralizedLinearEstimator(family='binomial')
# logistic_model.train(x=training_cols,y='reordered',training_frame = exp_data,fold_column='fold_no')

In [12]:
train_data = exp_data[exp_data['fold_no'] != 2]
test_data = exp_data[exp_data['fold_no'] == 2]

In [13]:
reordered_array = test_data['reordered'].as_data_frame()
r_array = train_data['reordered'].as_data_frame()

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [86]:
class ModelTrainer:
    
    def __init__(self,experiment_name,train_set,test_set=None):
        self.train_set = train_set
        self.test_set = test_set
        self.exp_name = experiment_name
        mlflow.set_experiment(self.exp_name)

    def __log_details(self,y_true,preds,prev_commit_hash,params):
        #log params 
        if params!=None:
                mlflow.log_params(params)
        else:
                mlflow.log_param("params",None)
                
        if self.test_set is not None:  
            
            #log metrics 
            pred_logits = [1 if pred >= 0.5 else 0 for pred in preds]
            mlflow.log_metric("precision",precision_score(y_true,pred_logits))
            mlflow.log_metric("recall",recall_score(y_true,pred_logits))
            mlflow.log_metric("f1",f1_score(y_true,pred_logits))
            mlflow.log_metric("AUC",roc_auc_score(y_true,preds))
            mlflow.log_metric("logloss",log_loss(y_true,preds))
            
        # log script url with version 
        commit_url = "https://github.com/d-sutariya/instacart_next_basket_prediction/tree/" + prev_commit_hash
        mlflow.log_param("repository url",commit_url)

        # log environment 
        os.system("conda env export > conda.yaml")
        mlflow.log_artifact("conda.yaml")

        # log dataset 
        dataset_path = "https://www.kaggle.com/datasets/deepsutariya/instacart-exp-data" 
        mlflow.log_param("dataset url",dataset_path)

        mlflow.end_run()
    
    def train_h2o_glm(self,prev_commit_hash,params=None):
        
        h2o_logistic_model = H2OGeneralizedLinearEstimator(family='binomial') \
                            .train(x=self.train_set.drop("reordered").columns,y='reordered',training_frame=self.train_set)
        # log the important stuffs
        with mlflow.start_run():
            
            mlflow.h2o.log_model(h2o_logistic_model,"h2o_logistic_model")
            
            # log perameters
            mlflow.log_param("family","binomial")
            mlflow.log_param("alpha",h2o_logistic_model.get_params()['alpha'])
            mlflow.log_param("lambda",h2o_logistic_model.get_params()['lambda'])

            preds = h2o_logistic_model.predict(self.test_set).as_data_frame()['p1']
            y_true = self.test_set['reordered'].as_data_frame()['reordered']

            # log dataset path,script path, environment
            self.__log_details(y_true,preds,prev_commit_hash,params)         
            
            del y_true
            del preds 
        
        return h2o_logistic_model

    def train_h2o_gbm(self,prev_commit_hash,params=None):

        if params != None:
            if "distribution" not in params.keys():
                params['distribution'] = 'bernoulli'
            
            start = time.time()
                
            h2o_gbm = H2OGradientBoostingEstimator(**params) \
                    .train(x=self.train_set.drop("reordered").columns,y='reordered',training_frame = self.train_set)
        else:
            
            start = time.time()
            h2o_gbm = H2OGradientBoostingEstimator() \
                    .train(x=self.train_set.drop("reordered").columns,y='reordered',training_frame = self.train_set)
            
        duration = get_time(start)
        
        with mlflow.start_run():
            # log model
            mlflow.h2o.log_model(h2o_gbm,"h2o_gbm_model")
            
            # log params
            if params!=None:
                mlflow.log_params(params)
            else:
                mlflow.log_param("params",None)
                
            mlflow.log_param("training_time",duration)
            
            preds = h2o_gbm.predict(self.test_set).as_data_frame()['p1']
            y_true = self.test_set['reordered'].as_data_frame()['reordered']

            # log evaluation metrics , dataset path,script path, environment
            self.__log_details(y_true,preds,prev_commit_hash,params) 

        del y_true,preds
        return h2o_gbm
    
    def train_xgb_rf(self,prev_commit_hash,params=None):
        
        dtrain = xgb.DMatrix(self.train_set, label = self.train_set['reordered'])
        dtest = xgb.DMatrix(self.test_set, label = self.test_set['reordered'])
        
        if params != None:
            
            if 'booster' not in params.keys():
                params['booster'] = 'gbtree'
                
            start = time.time()
            xgb_rf = xgb.train(params = params,dtrain=dtrain,num_boost_round=1)

        else :
            params = {
                'booster':'gbtree',
                'objective':'binary:logistic'
            }
            
            start = time.time()
            xgb_rf = xgb.train(params = params,dtrain=dtrain,num_boost_round=1)
            
        duration = get_time(start)

        with mlflow.start_run():

            mlflow.xgboost.log_model(xgb_rf,"xgb_rf_model")
            
            mlflow.log_param("training_time",duration)

            preds = xgb_rf.predict(dtest)
            y_true = self.test_set['reordered']

            self.__log_details(y_true,preds,prev_commit_hash,params)
            
        del y_true,preds
        return xgb_rf

    def train_xgb_gbm(self,prev_commit_hash,params=None):
        
        dtrain = xgb.DMatrix(self.train_set, label = self.train_set['reordered'])
        dtest = xgb.DMatrix(self.test_set, label = self.test_set['reordered'])
        
        if params != None:
            
            if 'booster' not in params.keys():
                params['booster'] = 'gbtree'
                
            start = time.time()
            xgb_gbm = xgb.train(params = params,dtrain=dtrain,num_boost_round=1)

        else :
            params = {
                'booster':'gbtree',
                'objective':'binary:logistic'
            }
            
            start = time.time()
            xgb_gbm = xgb.train(params = params,dtrain=dtrain)
            
        duration = get_time(start)

        with mlflow.start_run():

            mlflow.xgboost.log_model(xgb_gbm,"xgb_gbm_model")
            
            mlflow.log_param("training_time",duration)

            preds = xgb_gbm.predict(dtest)
            y_true = self.test_set['reordered']

            self.__log_details(y_true,preds,prev_commit_hash,params)
            
        del y_true,preds
        return xgb_gbm

    def train_lgb_gbm(self,prev_commit_hash,params=None)

In [60]:
model_trainer = ModelTrainer("instacart_data_check",train_data.drop("fold_no"),test_data.drop("fold_no"))

In [61]:
model = model_trainer.train_h2o_gbm("c7ef917635e4407d9b07297214ac80899a3018cb")

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


2024/10/18 12:55:59 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpcuq3pozg/model, flavor: h2o). Fall back to return ['h2o==3.46.0.5']. Set logging level to DEBUG to see the full traceback. 
2024/10/18 12:55:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
2024/10/18 12:57:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run judicious-duck-411 at: https://dagshub.com/tnbmarketplace/instacart_basket_analysis_exp_tracking.mlflow/#/experiments/0/runs/c05d261679e

In [50]:
cars = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/junit/cars_20mpg.csv")

# set the predictor names and the response column name
predictors = ["displacement","power","weight","acceleration","year"]
response = "cylinders"



Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [51]:
binary_array = np.random.randint(0, 2, cars.nrows)

# Convert the NumPy array to an H2OFrame
binary_column = h2o.H2OFrame(binary_array)

# Name the new column (optional)
binary_column.set_names(['reordered'])

# Append the binary column to the existing data
cars =cars.cbind(binary_column)
cars['reordered'] = cars['reordered'].asfactor()


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [87]:

xgb_model_trainer = ModelTrainer("cars_dummy_data",train.drop("name",axis=1),valid.drop("name",axis=1))
# train a GBM model


xgb_gbm = xgb_model_trainer.train_xgb_gbm("c7ef917635e4407d9b07297214ac80899a3018cb",params)


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [13:46:37] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [13:46:37] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
2024/10/18 13:46:40 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpxxtx3yro/model, flavor: xgboost). Fall back to return ['xgboost==2.0.3']. Set logging level to DEBUG to see the full traceback. 
2024/10/18 13:46:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/18 13:

In [ ]:
# training xgboost
train_pd_df = train_data.as_data_frame()
test_pd_df = test_data.as_data_frame()

xgb_model_trainer = ModelTrainer(train_pd_df.drop("fold_no",axis=1),test_pd_df.drop("fold_no",axis=1))
params = {
    'booster':'gbtree',
    'objective':'binary:logistic'
}
xgb_rf = xgb_model_trainer.train_xgb_rf("c7ef917635e4407d9b07297214ac80899a3018cb",params)

In [59]:
cars_gbm = model_trainer.train_h2o_gbm('avcd',params)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


2024/10/18 12:46:19 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpsjv7uys5/model, flavor: h2o). Fall back to return ['h2o==3.46.0.5']. Set logging level to DEBUG to see the full traceback. 
2024/10/18 12:46:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
2024/10/18 12:47:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run puzzled-conch-477 at: https://dagshub.com/tnbmarketplace/instacart_basket_analysis_exp_tracking.mlflow/#/experiments/1/runs/233c2f98a752